In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import wilcoxon
from scipy.stats import binomtest
from statsmodels.stats.multitest import multipletests #para corrección bonferroni
from scipy.stats import ttest_ind #prueba t para dos muestras independientes 
from itertools import  combinations #combinaciones para prueba t de comparaciones multiples
from scipy.stats import ttest_rel #Prueba t para muestras pareadas o no independientes 
from scipy.stats import mannwhitneyu #Prueba U de Mann-Whitney 
from scipy.stats import median_abs_deviation

def IQR(tabla,variable):
    '''Calcula el rango intercuartil (IRQ) de una serie de datos de un dataframe
    variable: str, nombre de la columna o fila del data frame que se quiere calcular el IQR'''
    try:
        variable = str(variable)
    except TypeError:
        raise TypeError('El argumento debe ser un string')
    Q1=tabla[variable].quantile(0.25)
    Q2=tabla[variable].quantile(0.5)
    Q3=tabla[variable].quantile(0.75)
    return Q3-Q1

In [17]:
#Cargar el archivo al computador y crear un dataframe 
data=pd.read_csv('FLIR_groups1and2.csv', sep=';')
# Seprar los datos en tablas para su posterior análisis
tabla1 = data.iloc[:, 2:30].copy()
tabla2 = data.iloc[:, 30:59].copy()
tabla3 = data.iloc[:, 58:86].copy()
tabla4 = data.iloc[:, 114:125].copy()
tabla0 = data.iloc[:, 0:1].copy()
tablaMAX = data.loc[:,['Unnamed: 3','Unnamed: 31','Unnamed: 59','Unnamed: 87']]
# print(tabla0.keys())
# print(tabla1.keys())
# print(tabla2.keys())
# print(tabla3.keys())
# print(tabla4.keys())
tablaMAX = tablaMAX.drop([0, 1])
tablaMAX = tablaMAX.apply(pd.to_numeric, errors='coerce')
tablaMAX.fillna(tablaMAX, inplace=True)
# Cambiamos los indices de la primera columna 
tablaMAX[''] = range(1, len(tablaMAX) + 1)  # Add a new column with the range
tablaMAX.set_index('', inplace=True)       # Set the new column as the index

TablaAnalisis = pd.concat([tabla0,tabla4], axis=1)
TablaAnalisis = TablaAnalisis.rename(columns={
    'Unnamed: 0': 'SubjectID', 
    'Unnamed: 115': 'aveOralM',
    'Unnamed: 116': 'Gender',
    'Unnamed: 117': 'Age',
    'Unnamed: 118': 'Ethnicity',
    'Unnamed: 119': 'T_atm',
    'Unnamed: 120': 'Humidity',
    'Unnamed: 122': 'Cosmetics'  # Fixed syntax - removed extra 'Unnamed 123'
})
TablaAnalisis.pop('Other parameters: ')
#Eliminamos las dos primeras filas que no contienen datos relevantes
TablaAnalisis = TablaAnalisis.drop([0, 1])
TablaAnalisis[''] = range(1, len(tablaMAX) + 1)  # Add a new column with the range
TablaAnalisis.set_index('', inplace=True)  
TablaAnalisis["ProMax"] = tablaMAX.mean(axis=1)

print(TablaAnalisis.keys())
TablaFrecuencias=pd.DataFrame()
TablaFrecuencias.index =['Media','Mediana','Moda','SD','MAD','Varianza','IQR','CV','CMV']



Index(['SubjectID', 'aveOralM', 'Gender', 'Age', 'Ethnicity', 'T_atm',
       'Humidity', 'Unnamed: 121', 'Cosmetics', 'Unnamed: 123', 'Unnamed: 124',
       'ProMax'],
      dtype='object')


In [18]:
# AVE ORAL M
TablaAnalisis['aveOralM'] = pd.to_numeric(TablaAnalisis['aveOralM'], errors='coerce')

MediaAve = TablaAnalisis['aveOralM'].mean()
MedianaAve = TablaAnalisis['aveOralM'].median()
ModaAve = TablaAnalisis['aveOralM'].mode()
SDAve = TablaAnalisis['aveOralM'].std()
MADAve=median_abs_deviation(TablaAnalisis['aveOralM'], nan_policy='omit') #desviación media absoluta
VARAve=TablaAnalisis['aveOralM'].var() #varianza
IQRave=IQR(TablaAnalisis,'aveOralM') #rango intercuartil
CVave= (SDAve/MediaAve)*100 
CVMave=(MADAve/MediaAve)*100


TablaFrecuencias['aveOralM']=MediaAve,MedianaAve,ModaAve.iloc[0],SDAve,MADAve,VARAve,IQRave,CVave,CVMave

In [20]:
#Genero
# Convertir valores de 'Gender' a binarios: Male -> 1, Female -> 0
TablaAnalisis['Gender']= TablaAnalisis['Gender'].replace({'Male': 1, 'Female': 0})
# MedianaGenero=TablaAnalisis['Gender'].median(axis=0)
ModaGenero=TablaAnalisis['Gender'].mode()
TablaFrecuencias.loc['Moda', 'Genero'] = ModaGenero.iloc[0]



In [ ]:
#AGE
ModaAge=TablaAnalisis['Age'].mode()
TablaFrecuencias.loc['Moda','Edad']=ModaAge.iloc[0]


In [ ]:
#Ethnicity

EtModa=TablaAnalisis['Ethnicity'].mode()
TablaFrecuencias.loc['Moda','Etnicidad']=EtModa.iloc[0]

In [23]:
#T_atm
TablaAnalisis['T_atm'] = pd.to_numeric(TablaAnalisis['T_atm'], errors='coerce')
TMedia=TablaAnalisis['T_atm'].mean()
Tmediana=TablaAnalisis['T_atm'].median()
TModa=TablaAnalisis['T_atm'].mode()
TSD=TablaAnalisis['T_atm'].std()
TMAD=median_abs_deviation(TablaAnalisis['T_atm'], nan_policy='omit') #desviación media absoluta
TVar=TablaAnalisis['T_atm'].var() #varianza
TIQR=IQR(TablaAnalisis,'T_atm') #rango intercuartil
TVC= (TSD/TMedia)*100
TVCM=(TMAD/TMedia)*100
TablaFrecuencias['T_atm']=TMedia,Tmediana,TModa.iloc[0],TSD,TMAD,TVar,TIQR,TVC,TVCM

In [24]:
#Humedad
TablaAnalisis['Humidity'] = pd.to_numeric(TablaAnalisis['Humidity'], errors='coerce')
HuMedia= TablaAnalisis['Humidity'].mean()
HuMediana= TablaAnalisis['Humidity'].median()
HuModa=TablaAnalisis['Humidity'].mode()
HuSD=TablaAnalisis['Humidity'].std()
HuMAD=median_abs_deviation(TablaAnalisis['Humidity'], nan_policy='omit')
HuVar=TablaAnalisis['Humidity'].var()
HuIQR=IQR(TablaAnalisis,'Humidity') #Rango intercuartil
HuCV= (HuSD/HuMedia)*100
HuCVM=(HuMAD/HuMedia)*100
TablaFrecuencias['Humidity'] = HuMedia,HuMediana,HuModa.iloc[0],HuSD,HuMAD,HuVar,HuIQR,HuCV,HuCVM

In [ ]:
#Cosmetics
TablaAnalisis['Cosmetics'] = pd.to_numeric(TablaAnalisis['Cosmetics'], errors='coerce')
CosModa=TablaAnalisis['Cosmetics'].mode()
CosMediana=TablaAnalisis['Cosmetics'].median()
CosVar=TablaAnalisis['Cosmetics'].var()
TablaFrecuencias.loc['Moda','Cosmetics']=CosModa.iloc[0]


In [15]:
#ProMAX
ProMAXmedia=TablaAnalisis['ProMax'].mean()
ProMAXmediana=TablaAnalisis['ProMax'].median()
ProMAXmoda=TablaAnalisis['ProMax'].mode()
ProMAXSD=TablaAnalisis['ProMax'].std()
ProMAXMAD=median_abs_deviation(TablaAnalisis['ProMax'], nan_policy='omit')
ProMAXVar=TablaAnalisis['ProMax'].var()
ProMAXIQR=IQR(TablaAnalisis,'ProMax') #Rango intercuartil
ProMAXCV= (ProMAXSD/ProMAXmedia)*100
ProMAXCVM=(ProMAXMAD/ProMAXmedia)*100
TablaFrecuencias['ProMAX']=ProMAXmedia,ProMAXmediana,ProMAXmoda.iloc[0],ProMAXSD,ProMAXMAD,ProMAXVar,ProMAXIQR,ProMAXCV,ProMAXCVM